# GAME OF THRONES: A GRAPH EXPERIMENT
## get the extracted characters informations from pickles
The dictionaries are loaded directly

In [7]:
import pickle
with open('characters.pickle','rb') as fp:
    characters = pickle.load(fp)

#characters

In [10]:
with open('characters_links.pickle','rb') as fp:
    character_links = pickle.load(fp)

#character_links

# Read the chapters and the book they belong to
The books have been cut into chapters themselves into a book directory named GOT{x} where x is the book in order of publication. We read the whole directory to retain only text files. we create a chapter list containing the text of the chapter and the book index

In [11]:
import os
from os.path import join

chapters = []

for root, dirs, files in os.walk('Downloads/US01-test-master/data'):
    for name in files:
        if name.endswith('txt'):
            book = int(root.split('/')[-1][-1:])
            with open(join(root,name),'r') as fp:
                chapters.append((book,fp.read()))
book               
print('found',len(chapters),'chapters')
#print(chapters[1])

found 345 chapters


# For each Chapter, we analyze the entities and prepare the sentences¶
Each chapter contains some Persons, that might be characters. We save them and their position together with the book they belong to. This will allow us to

- get a distance between persons

- disambiguate the characters using the book they appear in to

In [12]:
def getCandidates(name, shortened, key, equal=True):
    candidates=[]
    for s in shortened:
        if key != 'aliases':
            if equal:
                if name == s[key]:
                    candidates.append(s)
                elif name in s[key]:
                    candidates.append(s)
                
    if len(candidates) > 0:
        return candidates[0]['common_name']
    
    else:
        None
                 
def lemmatize(name, book):
    #add a character to the list if he is in the same book according to the 
    #function argument and wiki
    shortened=[]
    for c in characters:
        if c['books'] is not None:
            if book in c['books']:
                shortened.append(c)
                
    candidate = getCandidates(name, shortened, 'common_name')
    if candidate is not None :
        return candidate
        
    candidate = getCandidates(name, shortened, 'short_name')
    if candidate is not None :
        return candidate
        
    candidate = getCandidates(name, shortened, 'fullname')
    if candidate is not None :
        return candidate
        
    candidate = getCandidates(name, shortened, 'alias')
    if candidate is not None :
        return candidate
            
                
    
    return None

In [13]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import networkx as nx

STOPLIST = set(["n't", "'s", "'m", "ca"] + list(STOP_WORDS))
#SYMBOLS = " ".join(string.punctuation).split(" ") + ["-----", "---", "...", "“", "”", '\t','\n']
nlp = spacy.load('en_core_web_sm')

characters = {}
sentences = [] 
for i in range(len(chapters)):
    if i > 2:
        break
    book, chapter = chapters[i]
    chapter = chapter.strip().replace("\n", " ").replace("\t", " ")
    doc = nlp(chapter)

    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            entity = ent.text.replace("'s",'')
            if ent.text not in characters.keys():
                characters[entity] = {'books':set(),'positions':[]}
            
            characters[entity]['books'].add(book)
            characters[entity]['positions'].append((ent.start, ent.end))
            ent.merge(ent.root.tag_,ent.text, ent.label_)

#Remove useless thing such as punctuation

    sentence = []
    for token in doc:
       #print(token.ent_type_, token.text, token.pos_)
        if token.ent_type_ == 'PERSON':
            text = token.text.replace(' ', '_').replace("'s",'')
            tag = token.ent_type_
            sentence.append('%s|%s' % (text, tag))
        elif token.pos_ not in ['PUNCT','SPACE']:
            sentence.append(token.lemma_.strip() if token.lemma_ != "-PRON-" else token.lower_)
        elif token.pos_ == 'PUNCT' and token.text == '.':
            sentences.append(sentence)
            sentences.append([word for word in sentence if word not in STOPLIST])
            sentence = []

print(characters.keys())

dict_keys(['Waymar Royce', 'Gared', 'Royce', 'Ser Waymar Royce', 'Ser Waymar', 'Sworn', 'Bet', 'Mormont', 'Mallister', 'Mallisters', 'Will', 'Frost', 'Maester Aemon', 'Twilight', 'Down', 'Pale', 'Waymar', 'Tall', 'Robert', 'Would Gared', 'Bran', 'Mance Rayder', 'Robb', 'Jon', 'Eddard Stark', 'Jory Cassel', 'Eddard', 'Jon Snow', 'Ass', 'Race', 'Targaryen', 'Starks', 'Jory', 'Harwin', 'Stark', 'Snow', 'Desmond', 'Lady Stark', 'Something', 'Catelyn Tully Stark', 'Uncle Benjen', 'Ned', 'Grey Wind', 'Arya', 'Ghost', 'Septa Mordane', 'Mikken', 'Pentos', 'Myr', 'Run', 'Uncle Ben'])


In [15]:
print(lemmatize('Ned',2))

TypeError: string indices must be integers

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences, size=50, window=5, min_count=1, workers=4)
model.wv.most_similar('Bran|PERSON')

In [ ]:
import itertools
couples = list(itertools.combinations(characters, 2))
G.add_edges_from(couples)

In [ ]:
print(nx.info(G))
nx.write_gexf(G, 'one-chapter.gexf')